In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Tuple

In [ ]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(f'{len(text)}')

In [ ]:
print(text[:1000])

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocabulary ({vocab_size} elements): {''.join(chars)}")

In [ ]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [ ]:
# Encoder (str --> list[int])
encode = lambda s: [stoi[c] for c in s]

# Decoder (list[int] --> str)
decode = lambda l: ''.join([itos[i] for i in l])
ex: str = 'test de texte.'
print(encode(ex))
print(decode(encode(ex)))

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # tensor representation of the 1000 first data

## Split train/val

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [ ]:
block_size = 8
train_data[: block_size + 1]

In [ ]:
x = train_data[: block_size]
y = train_data[1: block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context}, the  target is: {target}')

In [ ]:
torch.manual_seed(1337)
batch_size: int = 4

def get_batch(split: str, debug: bool = False) -> Tuple[torch.Tensor, torch.Tensor]:
    data = train_data if split == 'train' else valid_data
    # Get random values between 0 and [size dataset - block_size], with shape (bs, )
    idx = torch.randint(len(data) - block_size, (batch_size, ))
    # This is the starting position in 'data'
    if debug:
        print(f'{idx=}')
        print(f'This should exactly be the first value of the first element in "x" tensor (24 with 1337 as seed) -> {data[idx[0]]}')
        print(f'This is the LAST value of the first element in "y" tensor.{data[idx[0] + block_size]}')
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i+1 : i + block_size + 1] for i in idx])

    return x, y

In [ ]:
xb, yb = get_batch(split='train')
print(f'inputs:\n{xb.shape}\n{xb}')
print(f'targets:\n{yb.shape}\n{yb}')

In [ ]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target: {target}")

In [ ]:
print(xb) # our input to the transformer

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table: nn.Embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx: torch.Tensor, targets: torch.Tensor | None = None) -> Tuple[torch.Tensor, torch.Tensor]:
        # shape (bs, t, c) where bs is batch size 4, t is temporal dimension 8, and c is channel dimension = vocab_size = 65
        logits: torch.Tensor = self.token_embedding_table(idx)

        if targets is None:
            loss = None
            return logits, loss

        bs, t, c = logits.shape
        # shape (bs*t, c) = (32, 65)
        logits = logits.view(bs * t, c)
        # from shape (bs, t) to (bs*t) = (32)
        targets = targets.view(bs * t)

        # shape (
        loss: torch.Tensor = F.cross_entropy(logits, targets)

        return logits, loss

    # continues the generation for a batch, in the time dimension
    def generate(self, idx: torch.Tensor, max_new_tokens: int):
        # idx has shape (b, t), and is an array of indices in the current context
        for _ in range(max_new_tokens):
            # get the preds
            logits, loss = self(idx)

            # keep last time step only
            # shape (b, c)
            logits = logits[:, -1, :]
            # shape (b, c)
            probs = F.softmax(logits, dim=-1)

            # sample from distribution
            # shape (b, 1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence -> concatenate along time axis
            # shape (b, t+1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
m = BigramLanguageModel(vocab_size=vocab_size)
results: Tuple[torch.Tensor, torch.Tensor] = m(xb, yb)

In [ ]:
print(f'loss: {results[1]=} | shape logits: {results[0].shape}')

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long)
# index into 0th row -> we work with batches and only get the first one
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size: int = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # eval the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))

# Self-Attention: the mathematical trick

In [ ]:
torch.manual_seed(1337)
# batch, time, channels
b, t, c = 4, 8, 2
x = torch.rand(b, t, c)
x.shape

In [ ]:
# We want x[b, t] = mean_i{i<=t} x[b,i]
# bow : bag of words
x_bow = torch.zeros((b, t, c))

# iterate over batch dimension
for _b in range(b):
    # iterate over time dimension
    for _t in range(t):
        # (_t, c)
        x_prev = x[_b, :_t+1]
        x_bow[_b, _t] = torch.mean(x_prev, 0)